In [1]:
import pandas as pd
import metnum
import numpy as np
import utils
import seaborn as sns
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup

df_completo = pd.read_csv('../data/train.csv')
df_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 23 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          240000 non-null  int64  
 1   titulo                      234613 non-null  object 
 2   descripcion                 238381 non-null  object 
 3   tipodepropiedad             239954 non-null  object 
 4   direccion                   186928 non-null  object 
 5   ciudad                      239628 non-null  object 
 6   provincia                   239845 non-null  object 
 7   antiguedad                  196445 non-null  float64
 8   habitaciones                217529 non-null  float64
 9   garages                     202235 non-null  float64
 10  banos                       213779 non-null  float64
 11  metroscubiertos             222600 non-null  float64
 12  metrostotales               188533 non-null  float64
 13  idzona        

In [ ]:
df_completo['ciudad'].value_counts()

Elijo las ciudades con mas datos

In [ ]:
df_quer = df_completo[df_completo['ciudad']=='Querétaro']
df_BJ = df_completo[df_completo['ciudad']=='Benito Juárez']
df_zap = df_completo[df_completo['ciudad']=='Zapopan']
df_SLP = df_completo[df_completo['ciudad']=='San Luis Potosí']

df_BJ['precio'].mean() 


Elijo la ciudad con mayor datos y veo la relacion con precio de algunas propiedades

In [ ]:
props = ['antiguedad', 'habitaciones', 'garages', 'banos', 'metrostotales', 'metroscubiertos']
pred = 'precio'

In [ ]:
utils.graficarRelacionEntrePropiedades(props, df_quer, pred, sigma=3)

In [22]:
propGroup = 'ciudad'
groups = [
    'Querétaro',
    'Benito Juárez',
    'Zapopan',
    'San Luis Potosí'
]

props = [
    'metroscubiertos', 
#     'antiguedad', 
#     'piscina', 
#     'centroscomercialescercanos', 
#     'usosmultiples', 
#     'gimnasio', 
#     'escuelascercanas'
]
phis = [ 
        lambda x: x, #metros2
#         lambda x: (1/(x+1)), #antiguo
#         lambda x: (x+1)**2, #pileta
#         lambda x: (x+1), #shopping
#         lambda x: (x+1)**2, #SUM
#         lambda x: (x+1)**2, #gimnasio
#         lambda x: (x+1) #escuelas
]

Veo las precisiones en las principales ciudades

In [23]:
for i in range(len(groups)):
    df_group = df_completo[df_completo[propGroup]==groups[i]]
    val = utils.predictWithKFold(df_group, phis, props)
    print(f"En {groups[i]} erramos por {np.round(val, decimals = 3)}")

En Querétaro erramos por 0.351
En Benito Juárez erramos por 0.436
En Zapopan erramos por 0.45
En San Luis Potosí erramos por 0.374


Para la ciudad Querétaro voy a ver cuanto cambia la precision al aumentar las variables que uso

In [ ]:
for i in range(len(props)):
    print(props[:i+1])
    val = utils.predictWithKFold(df_quer, phis, props[:i+1])
    print(f"Precision para {i} variables es de {val}")

Quiero ver si solo importa la cantidad de variables o si cada una aporta distinto(seguramente que cada una aporta distinto)

In [ ]:
for i in range(len(props)):
    randomIndex = np.random.choice(range(len(props)), i+1, replace=False)
    propsR = list(np.array(props)[randomIndex])
    print(propsR)
    phisR = list(np.array(phis)[randomIndex])
    val = utils.predictWithKFold(df_quer, phisR, propsR)
    print(f"Precision para {i} variables aleatorias es de {val}")

# Incluyendo palabrasCaras

In [20]:

palabrasCaras = [('completo',6), ('servicio',2), ('jardín', 3), ('piscina', 9), ('granito', 1), ('acabados',2), ('cochera',3), ('amplia', 6), ('alberca', 5), ('garage', 4), ('excelente', 10), ('porcelanato', 3), ('amplio', 6), ('lujo', 10), ('equipada', 4), ('garaje', 4), ('residencial', 1), ('mármol', 3), ('cristal', 2), ('calentador', 2), ('balcón', 5), ('iluminación', 6), ('riego', 10), ('hermosa', 10), ('hermoso', 10), ('vigilancia', 9), ('seguridad', 3), ('amplios', 3), ('tv', 1), ('palapa', 2), ('gimnasio', 7), ('completos', 3), ('jacuzzi', 10), ('acondicionado', 2), ('acondicionados', 3), ('minisplit', 8)]
palabrasCaras2 = [('infonavit', 0.001),('completo',6), ('piscina', 9), ('cochera',5), ('amplia', 6), ('alberca', 2), ('garage', 4), ('excelente', 10), ('amplio', 6), ('lujo', 10), ('garaje', 4), ('balcón', 5), ('iluminación', 6), ('hermosa', 10), ('hermoso', 10), ('vigilancia', 9), ('seguridad', 3), ('amplios', 5), ('gimnasio', 7), ('jacuzzi', 10), ('minisplit', 8)]


palabrasCarasConValores = [('ventanas', 2),('golf', 10),('vitropiso', 3),('grandes', 5),('panorámica', 6),('moderno', 4),('amenidades', 9),('solares', 7),('elevador', 4),('antecomedor', 3),('transporte', 8),('pequeña', 0.0001),('inmejorable', 6),('hospitales', 5),('infonavit', 0.0001),('completo',6), ('servicio',2), ('jardín', 3), ('piscina', 9), ('granito', 1), ('acabados',2), ('cochera',3), ('amplia', 6), ('alberca', 5), ('garage', 4), ('excelente', 10), ('excelentes', 10), ('porcelanato', 3), ('amplio', 6), ('lujo', 10), ('equipada', 4), ('garaje', 4), ('residencial', 1), ('mármol', 3), ('cristal', 2), ('calentador', 2), ('balcón', 5), ('iluminación', 6), ('riego', 10), ('hermosa', 10), ('hermoso', 10), ('vigilancia', 9), ('seguridad', 3), ('amplios', 3), ('tv', 1), ('palapa', 2), ('gimnasio', 7), ('completos', 3), ('jacuzzi', 10), ('acondicionado', 2), ('acondicionados', 3), ('minisplit', 8), ('preciosa', 8), ('nuevo', 5), ('grande', 7), ('techado', 3), ('techada', 3), ('gravamen', 0.0001), ('avenida', 5), ('crédito', 0.0001), ('remodelada', 8), ('ideal', 7), ('eventos', 4), ('parque', 6), ('bonita', 10), ('bonito', 10), ('precio', 0.0001), ('integral', 4)]

len(palabrasCarasConValores)

68

In [24]:
props = [
    'metroscubiertos', 
#     'antiguedad', 
#     'piscina', 
#     'centroscomercialescercanos', 
#     'usosmultiples', 
#     'gimnasio', 
#     'escuelascercanas',
    'valor'
]
phis = [ 
        lambda x: x, #metros2
#         lambda x: (1/(x+1)), #antiguo
#         lambda x: (x+1)**2, #pileta
#         lambda x: (x+1), #shopping
#         lambda x: (x+1)**2, #SUM
#         lambda x: (x+1)**2, #gimnasio
#         lambda x: (x+1), #escuelas
        lambda x: (x+1) #valor
]


def getValor(desc, palabra):
    regexPattern = re.compile(palabra)
    soup = BeautifulSoup(desc)
    desc = soup.get_text()
    listOfmatches = regexPattern.findall(desc)
    return len(listOfmatches)
# A['valor'] = A['descripcion'].apply(getValor)


for group in groups:
    A = df_completo[df_completo['ciudad'] == group]
    A = A.dropna(subset=['descripcion'])
    A['valor'] = 0

    for (palabra, mult) in palabrasCarasConValores:
        A['valor'] += A.apply(lambda x : mult * getValor(x['descripcion'], palabra), axis=1)
    
    print(f'Estimacion para {group}')
    print(utils.predictWithKFold(A, phis, props, validation=utils.RMSLE, k=5))
    print(utils.predictWithKFold(A, phis, props, validation=utils.RMSE, k=5))

Estimacion para Querétaro
0.3503555328684418
803278.9679159279
Estimacion para Benito Juárez
0.4133607056537126
1429539.7709557507
Estimacion para Zapopan
0.45455638174795227
1269302.0824983718
Estimacion para San Luis Potosí
0.37431170008983594
704666.560885717
